In [5]:
# First cell - clean everything
!pip uninstall torch torchtext -y
!pip cache purge

Files removed: 6 (188.8 MB)


In [9]:
!pip install torchtext==0.15.1

   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   -------------------------- ------------- 1.3/1.9 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 3.8 MB/s eta 0:00:00


In [10]:
# Third cell - verify installation
import torch
import torchtext
print(f"PyTorch: {torch.__version__}")  # Should show 1.8.0
print(f"TorchText: {torchtext.__version__}")  # Should show 0.9.0

PyTorch: 2.0.0+cpu
TorchText: 0.15.0


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

# from torchtext.legacy.data import Field, BucketIterator
from torchtext.datasets import Multi30k
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

from torch.utils.data import DataLoader

import numpy as np
import spacy
import random

from torch.utils.tensorboard import SummaryWriter


In [14]:
!python -m spacy download de_core_news_sm


     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
      --------------------------------------- 0.3/14.6 MB ? eta -:--:--
     -- ------------------------------------- 1.0/14.6 MB 3.9 MB/s eta 0:00:04
     ----- ---------------------------------- 1.8/14.6 MB 3.5 MB/s eta 0:00:04
     ------- -------------------------------- 2.6/14.6 MB 3.7 MB/s eta 0:00:04
     --------- ------------------------------ 3.4/14.6 MB 3.8 MB/s eta 0:00:03
     ----------- ---------------------------- 4.2/14.6 MB 3.8 MB/s eta 0:00:03
     ------------- -------------------------- 5.0/14.6 MB 3.8 MB/s eta 0:00:03
     --------------- ------------------------ 5.8/14.6 MB 3.8 MB/s eta 0:00:03
     ----------------- ---------------------- 6.6/14.6 MB 3.8 MB/s eta 0:00:03
     -------------------- ------------------- 7.3/14.6 MB 3.8 MB/s eta 0:00:02
     ---------------------- ----------------- 8.1/14.6 MB 3.8 MB/s eta 0:00:02
     ------------------------ --------------- 8.9/14.6 MB 3.8 MB/s

In [15]:
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     ---- ----------------------------------- 1.3/12.8 MB 4.2 MB/s eta 0:00:03
     ------ --------------------------------- 2.1/12.8 MB 3.9 MB/s eta 0:00:03
     --------- ------------------------------ 2.9/12.8 MB 4.0 MB/s eta 0:00:03
     ----------- ---------------------------- 3.7/12.8 MB 4.0 MB/s eta 0:00:03
     ------------- -------------------------- 4.5/12.8 MB 3.9 MB/s eta 0:00:03
     ----------------- ---------------------- 5.5/12.8 MB 3.9 MB/s eta 0:00:02
     ------------------- -------------------- 6.3/12.8 MB 3.9 MB/s eta 0:00:02
     ---------------------- ----------------- 7.1/12.8 MB 3.9 MB/s eta 0:00:02
     ------------------------ --------------- 7.9/12.8 MB 3.9 MB/s eta 0:00:02
     --------------------------- ------------ 8.7/12.8 MB 3.9 MB/s eta 0:00:02
     ----------------------------- ---------- 9.4/12.8 MB 3.9 MB/s

In [16]:
!pip uninstall torchtext portalocker -y
!pip cache purge

Found existing installation: torchtext 0.15.1
Uninstalling torchtext-0.15.1:
  Successfully uninstalled torchtext-0.15.1
Found existing installation: portalocker 3.2.0
Uninstalling portalocker-3.2.0:
  Successfully uninstalled portalocker-3.2.0
Files removed: 32 (205.2 MB)


In [17]:
!pip install torchtext==0.15.1 portalocker==2.7.0


   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/1.9 MB 4.2 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/1.9 MB 3.9 MB/s eta 0:00:01
   ------------------------------------- -- 1.8/1.9 MB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 3.3 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [torchtext]
   -------------------- ------------------- 1/2 [torchtext]
   -------------------- ------------------- 1/2 [torchtext]
   ---------------------------------------- 2/2 [torchtext]



In [7]:
!pip install datasets

   ---------------------------------------- 0.0/558.8 kB ? eta -:--:--
   ---------------------------------------- 558.8/558.8 kB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   - -------------------------------------- 0.8/26.2 MB 4.2 MB/s eta 0:00:07
   -- ------------------------------------- 1.6/26.2 MB 4.0 MB/s eta 0:00:07
   --- ------------------------------------ 2.4/26.2 MB 3.9 MB/s eta 0:00:07
   ---- ----------------------------------- 2.9/26.2 MB 3.9 MB/s eta 0:00:06
   ----- ---------------------------------- 3.9/26.2 MB 3.9 MB/s eta 0:00:06
   ------- -------------------------------- 4.7/26.2 MB 3.9 MB/s eta 0:00:06
   -------- ------------------------------- 5.8/26.2 MB 3.9 MB/s eta 0:00:06
   --------- ------------------------------ 6.3/26.2 MB 3.9 MB/s eta 0:00:06
   ---------- ----------------------------- 7.1/26.2 MB 3.8 MB/s eta 0:00:06
   ----------- ---------------------------- 7.9/26.2 MB 3.8 MB/s eta 0:00:05
   ------

In [41]:
from datasets import load_dataset

multi30k = load_dataset("bentrevett/multi30k")
train_data = multi30k["train"]
valid_data = multi30k["validation"]
test_data = multi30k["test"]

# Example: Print first 3 German-English pairs
for i in range(3):
    print(f"German: {train_data[i]['de']}")
    print(f"English: {train_data[i]['en']}\n")

German: Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.
English: Two young, White males are outside near many bushes.

German: Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.
English: Several men in hard hats are operating a giant pulley system.

German: Ein kleines Mädchen klettert in ein Spielhaus aus Holz.
English: A little girl climbing into a wooden playhouse.



In [42]:
import spacy
from datasets import load_dataset
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
import numpy as np
import random
from torch.utils.tensorboard import SummaryWriter

# Load tokenizers
spacy_ger = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

def tokenizer_de(text):
    return [tok.text for tok in spacy_ger.tokenizer(text)]

def tokenizer_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

# Test tokenizer
text = "Hallo, wie geht es dir?"
print(tokenizer_de(text))  # ['Hallo', ',', 'wie', 'geht', 'es', 'dir', '?']

# Special tokens
BOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'
PAD_TOKEN = '<pad>'
UNK_TOKEN = '<unk>'

# Load dataset from Hugging Face
multi30k = load_dataset("bentrevett/multi30k")
train_data = multi30k["train"]
valid_data = multi30k["validation"]
test_data = multi30k["test"]

# Build vocabulary
def yield_tokens(data, language='de'):
    for example in data:
        yield tokenizer_de(example['de']) if language == 'de' else tokenizer_en(example['en'])

# German vocabulary
german_vocab = build_vocab_from_iterator(
    yield_tokens(train_data, 'de'),
    max_tokens=10000,
    min_freq=2,
    specials=[PAD_TOKEN, BOS_TOKEN, EOS_TOKEN, UNK_TOKEN]
)
german_vocab.set_default_index(german_vocab[UNK_TOKEN])

# English vocabulary
english_vocab = build_vocab_from_iterator(
    yield_tokens(train_data, 'en'),
    max_tokens=10000,
    min_freq=2,
    specials=[PAD_TOKEN, BOS_TOKEN, EOS_TOKEN, UNK_TOKEN]
)
english_vocab.set_default_index(english_vocab[UNK_TOKEN])

# Text transformation functions
def german_transform(text):
    return [german_vocab[BOS_TOKEN]] + german_vocab(tokenizer_de(text)) + [german_vocab[EOS_TOKEN]]

def english_transform(text):
    return [english_vocab[BOS_TOKEN]] + english_vocab(tokenizer_en(text)) + [english_vocab[EOS_TOKEN]]

# Example usage
print(german_transform("Hallo, wie geht es dir?"))
print(english_transform("Hello, how are you?"))

['Hallo', ',', 'wie', 'geht', 'es', 'dir', '?']
[1, 3, 8, 170, 61, 301, 7286, 2515, 2]
[1, 3, 15, 889, 17, 1328, 2470, 2]


In [43]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout_value):
        super(Encoder,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout_value)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout_value)


    def forward(self, x):
        # x shape: (seq_length, N)

        embedding = self.dropout(self.embedding(x))
        #embedding shape: (seq_length, N, embedding_size)

        
        outputs, (hidden, cell) = self.rnn(embedding)

        return hidden, cell

### Hidden size of the Encoder and Decoder must be the same

In [44]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout_value):
        super(Decoder,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout_value)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout_value)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        #x shape: (N) -> we want (1, N)
        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x))
        #embedding shape: (1, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        #outputs shape: (1, N, hidden_size)
        predictions = self.fc(outputs)
        #predictions shape: (1, N, output_size)
        predictions = predictions.squeeze(0)
        #predictions shape: (N, output_size)
        return predictions, hidden, cell


In [45]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = self.decoder.fc.out_features

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(source.device)

        hidden, cell = self.encoder(source)
        x = target[0] #grab the first token from target

        for t in range(1, target_len):
            output, hidden, cell = self.decoder(x, hidden, cell)
            #output shape: (N, target_vocab_size)

            outputs[t] = output

            best_guess = output.argmax(1)

            x = target[t] if random.random() < teacher_forcing_ratio else best_guess

        return outputs


In [74]:
num_epochs = 20
learning_rate = 0.001
batch_size = 8


load_model = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size_encoder = len(german_vocab)
input_size_decoder = len(english_vocab)
output_size = len(english_vocab)
encoder_embedding_size = 128
decoder_embedding_size = 128
hidden_size = 256
num_layers = 1
encoder_dropout_value = 0.5
decoder_dropout_value = 0.5



In [75]:
writer = SummaryWriter(f'runs/Loss_plot')  
step = 0

In [76]:
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader

# Convert to MapStyleDataset for shuffling/sorting
train_dataset = to_map_style_dataset(train_data)
valid_dataset = to_map_style_dataset(valid_data)
test_dataset = to_map_style_dataset(test_data)

# Create custom collate function with sorting
def collate_batch(batch):
    src_batch, trg_batch = [], []
    for item in batch:
        # Access German text with item['de'] and English with item['en']
        src_batch.append(torch.tensor(german_transform(item['de'])))
        trg_batch.append(torch.tensor(english_transform(item['en'])))
    
    # Sort by source length (descending)
    sorted_indices = sorted(
        range(len(src_batch)),
        key=lambda i: len(src_batch[i]),
        reverse=True
    )
    
    src_sorted = [src_batch[i] for i in sorted_indices]
    trg_sorted = [trg_batch[i] for i in sorted_indices]
    
    # Pad sequences
    src_padded = pad_sequence(src_sorted, padding_value=german_vocab[PAD_TOKEN])
    trg_padded = pad_sequence(trg_sorted, padding_value=english_vocab[PAD_TOKEN])
    
    return src_padded, trg_padded

# Create DataLoaders with sorting
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_batch,
    num_workers=0
)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=batch_size,
    collate_fn=collate_batch,
    num_workers=0
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=collate_batch,
    num_workers=0
)

In [77]:
encoder_net = Encoder(input_size_encoder, encoder_embedding_size, hidden_size, num_layers, encoder_dropout_value).to(device)

c:\Users\YN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [78]:
decoder_net = Decoder(input_size_decoder, decoder_embedding_size, hidden_size,output_size, num_layers, decoder_dropout_value).to(device)

In [79]:
model = Seq2Seq(encoder_net, decoder_net).to(device)

In [80]:
criterion = nn.CrossEntropyLoss(ignore_index=english_vocab[PAD_TOKEN])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [81]:
def evaluate(model, loader):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for src, trg in loader:
            src, trg = src.to(device), trg.to(device)
            output = model(src, trg, teacher_forcing_ratio=0)  # No teacher forcing
            
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            loss = criterion(output, trg)
            total_loss += loss.item()
    
    return total_loss / len(loader)

In [82]:
import time
import math
from datetime import datetime, timedelta

# Initialize TensorBoard writer
writer = SummaryWriter('runs/translation_experiment')

# Training variables
best_valid_loss = float('inf')
patience = 0
max_patience = 3
total_batches = len(train_loader)
start_train_time = time.time()

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    model.train()
    train_loss = 0
    
    # Epoch header
    print(f"\n{'='*50}")
    print(f"Epoch {epoch+1}/{num_epochs} | Started: {datetime.now().strftime('%H:%M:%S')}")
    print(f"{'='*50}")
    
    # Training phase with progress bar
    for batch_idx, (src, trg) in enumerate(train_loader):
        batch_start_time = time.time()
        
        # Training steps
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model(src, trg)
        
        # Loss calculation
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()
        
        train_loss += loss.item()
        
        # Progress tracking
        current_time = time.time()
        elapsed_time = current_time - batch_start_time
        batches_done = batch_idx + 1
        batches_left = total_batches - batches_done
        avg_batch_time = (current_time - epoch_start_time) / batches_done
        remaining_time = avg_batch_time * batches_left
        
        # Print progress every 10% of batches (FIXED PARENTHESES)
        if (batch_idx + 1) % max(1, len(train_loader) // 10) == 0:
            print(f"[Batch {batches_done}/{total_batches}] "
                  f"Loss: {loss.item():.3f} | "
                  f"Avg: {avg_batch_time:.2f}s/batch | "
                  f"ETA: {str(timedelta(seconds=int(remaining_time)))}")
        
        # TensorBoard logging
        writer.add_scalar('Loss/train_batch', loss.item(), epoch * total_batches + batch_idx)
    
    # Validation phase
    valid_start_time = time.time()
    valid_loss = evaluate(model, valid_loader, criterion)
    valid_loss /= len(valid_loader)
    
    # Epoch statistics
    epoch_time = time.time() - epoch_start_time
    train_loss /= len(train_loader)
    
    # TensorBoard logging
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/valid', valid_loss, epoch)
    writer.add_scalar('Time/epoch', epoch_time, epoch)
    
    # Print epoch summary
    print(f"\n{'='*50}")
    print(f"Epoch {epoch+1} Summary:")
    print(f"  Training Loss: {train_loss:.3f} | PPL: {math.exp(train_loss):7.3f}")
    print(f"  Validation Loss: {valid_loss:.3f} | PPL: {math.exp(valid_loss):7.3f}")
    print(f"  Time: {str(timedelta(seconds=int(epoch_time)))}")
    print(f"  Total Elapsed: {str(timedelta(seconds=int(time.time() - start_train_time)))}")
    print(f"{'='*50}")
    
    # Early stopping check
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best_model.pt')
        patience = 0
        print("↗ New best model saved!")
    else:
        patience += 1
        if patience >= max_patience:
            print(f"Early stopping after {patience} epochs without improvement")
            break

writer.close()



Epoch 1/20 | Started: 16:47:42


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 4103168 bytes.